<a href="https://colab.research.google.com/github/kalvin-osoro/ml_gw/blob/main/office_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [697]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler

# first neural network with keras tutorial
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [698]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/ml/Nairobi Office Price Ex.csv"
data = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [699]:
data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER,SIZE,PRICE
0,karen,yes,serene,no,yes,yes,yes,yes,32.502345,31.707006
1,madaraka,yes,semi_serene,yes,yes,yes,no,no,53.426804,68.777596
2,karen,no,noisy,no,yes,yes,yes,yes,61.530358,62.562382
3,karen,yes,semi_serene,no,no,no,yes,yes,47.475640,71.546632
4,buruburu,no,semi_serene,no,yes,yes,yes,yes,59.813208,87.230925


In [700]:
data.shape

(14, 10)

In [701]:
data.dtypes

LOC           object
FUR           object
AMB           object
PROX_SCH      object
PROX_ROAD     object
PROX_MALL     object
WATER         object
HK_SER        object
SIZE         float64
PRICE        float64
dtype: object

#data encoding

In [702]:
original_features=list(data.columns)
original_features
len(original_features)

10

In [703]:
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER
0,karen,yes,serene,no,yes,yes,yes,yes
1,madaraka,yes,semi_serene,yes,yes,yes,no,no
2,karen,no,noisy,no,yes,yes,yes,yes
3,karen,yes,semi_serene,no,no,no,yes,yes
4,buruburu,no,semi_serene,no,yes,yes,yes,yes


In [704]:
cat_features=list(cat_data.columns)
cat_features

['LOC', 'FUR', 'AMB', 'PROX_SCH', 'PROX_ROAD', 'PROX_MALL', 'WATER', 'HK_SER']

In [705]:
nominal=['LOC','FUR','PROX_SCH','PROX_ROAD','PROX_MALL','WATER','HK_SER']
ordinal=list(set(cat_features)-set(nominal))
numerical=list(set(original_features)-set(cat_features))
target=['PRICE']
data[target]

,PRICE
0,31.707006
1,68.777596
2,62.562382
3,71.546632
4,87.230925
5,78.211518
6,79.641973
7,59.171489
8,75.331242
9,71.300880


In [706]:
for feature in ordinal:
  data[feature]=(data[feature].astype('category')).cat.codes
  

In [707]:
df_nominal=pd.get_dummies(data[nominal])
df_ordinal=data[ordinal]
df_numerical=data[numerical]
encoded_data=pd.concat([df_numerical,df_nominal,df_ordinal],axis=1)
# encoded_data=pd.concat([df_nominal,df_ordinal],axis=1)
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


In [708]:
encoded_data.shape

(14, 20)

normalization

In [709]:
# encoded_data[target]
df_numerical

,SIZE,PRICE
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925
5,55.142188,78.211518
6,52.211797,79.641973
7,39.299567,59.171489
8,48.105042,75.331242
9,52.550014,71.300880


In [710]:
num=df_numerical.to_numpy()
df_numerical

,SIZE,PRICE
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925
5,55.142188,78.211518
6,52.211797,79.641973
7,39.299567,59.171489
8,48.105042,75.331242
9,52.550014,71.300880


In [711]:
encoded_data.shape

(14, 20)

In [712]:
# columns_titles = ["PRICE","SIZE",'']
# encoded_data=encoded_data.reindex(columns=columns_titles)

In [713]:
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


In [714]:
num=StandardScaler().fit_transform(df_numerical)
# y=StandardScaler().fit_transform(y)
# y
num

array([[-2.28520557, -2.72264663],
       [ 0.40190153,  0.0124287 ],
       [ 1.44255533, -0.44613097],
       [-0.36234362,  0.21672874],
       [ 1.22203988,  1.37391891],
       [ 0.62219022,  0.70846532],
       [ 0.24587098,  0.81400455],
       [-1.41231026, -0.69631171],
       [-0.28151616,  0.495958  ],
       [ 0.28930471,  0.19859708],
       [-0.62636235, -0.99186131],
       [ 0.52066778,  1.02330965],
       [-0.78761614, -0.48696528],
       [ 1.01082367,  0.50050494]])

In [715]:
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


In [716]:
#Changing pandas dataframe to numpy array
# X = encoded_data.iloc[:,1:20].values
# y = data.iloc[:,:9].values

normalization

In [717]:
cols_to_norm = ['SIZE','PRICE']
encoded_data[cols_to_norm] = encoded_data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [718]:
df= encoded_data

In [719]:
df

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,0.000000,0.000000,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,0.720837,0.667651,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,1.000000,0.555713,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,0.515822,0.717522,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,0.940845,1.000000,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1
5,0.779931,0.837558,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,1,2
6,0.678980,0.863321,0,0,0,1,0,1,0,0,1,0,1,1,0,1,0,0,1,3
7,0.234161,0.494642,0,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,2
8,0.537505,0.785684,0,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,0,1
9,0.690632,0.713096,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,2


In [720]:
# Changing pandas dataframe to numpy array
X = df.iloc[:,1:20].values
# y = df.iloc[:,:1].values
y=df['PRICE']

In [721]:
X

array([[0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 2.        ],
       [0.66765082, 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 1.        , 0.        , 1.        ],
       [0.55571323, 0.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 0.        ],
       [0.71752187, 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        1.   

In [722]:
y

0     0.000000
1     0.667651
2     0.555713
3     0.717522
4     1.000000
5     0.837558
6     0.863321
7     0.494642
8     0.785684
9     0.713096
10    0.422497
11    0.914414
12    0.545745
13    0.786794
Name: PRICE, dtype: float64

In [723]:
df.shape

(14, 20)

create train/test

In [724]:

x_train, x_test, y_train, y_test = train_test_split(    
    X, y, test_size=0.25, random_state=42)

define the keras model

In [725]:
...

model = Sequential()
model.add(Dense(25, input_shape=(19,), activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
# model.add(Dense(1, activation='sigmoid'))

# model = Sequential()
# model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
# model.add(Dense(10, activation='relu')) # Hidden 2
# model.add(Dense(1)) # Output
# ...

compile the keras model

In [726]:

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto', 
                        restore_best_weights=True)



In [727]:


model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[
            monitor],verbose=2, batch_size=10, epochs=100)

# history = model.fit(X, y, batch_size=64, epochs=2, validation_split=0.2)

# test_scores = model.evaluate(x_test, y_test, verbose=2)
# print("Test loss:", test_scores[0])
# print("Test accuracy:", test_scores[1])



Epoch 1/100
1/1 - 0s - loss: 0.6249 - accuracy: 0.0000e+00 - val_loss: 0.4451 - val_accuracy: 0.2500 - 482ms/epoch - 482ms/step
Epoch 2/100
1/1 - 0s - loss: 0.5750 - accuracy: 0.0000e+00 - val_loss: 0.4172 - val_accuracy: 0.2500 - 34ms/epoch - 34ms/step
Epoch 3/100
1/1 - 0s - loss: 0.5268 - accuracy: 0.0000e+00 - val_loss: 0.3926 - val_accuracy: 0.2500 - 25ms/epoch - 25ms/step
Epoch 4/100
1/1 - 0s - loss: 0.4815 - accuracy: 0.0000e+00 - val_loss: 0.3710 - val_accuracy: 0.2500 - 31ms/epoch - 31ms/step
Epoch 5/100
1/1 - 0s - loss: 0.4400 - accuracy: 0.0000e+00 - val_loss: 0.3522 - val_accuracy: 0.2500 - 27ms/epoch - 27ms/step
Epoch 6/100
1/1 - 0s - loss: 0.4009 - accuracy: 0.0000e+00 - val_loss: 0.3354 - val_accuracy: 0.2500 - 27ms/epoch - 27ms/step
Epoch 7/100
1/1 - 0s - loss: 0.3644 - accuracy: 0.1000 - val_loss: 0.3211 - val_accuracy: 0.2500 - 31ms/epoch - 31ms/step
Epoch 8/100
1/1 - 0s - loss: 0.3294 - accuracy: 0.1000 - val_loss: 0.3096 - val_accuracy: 0.2500 - 25ms/epoch - 25ms/ste

In [728]:
# # fit the keras model on the dataset
# model.fit(X, y, epochs=10, batch_size=1)

evaluate the keras model

In [729]:

# evaluate the keras model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

1/1 [==============================] - 0s 19ms/step - loss: 0.2861 - accuracy: 0.2500
Accuracy: 25.00
